<a href="https://colab.research.google.com/github/jwshin0908/Algorithm/blob/main/KOREANRE_DT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=dd8ab1d46d1d8e86afbb9c0895e4f848fe44c681890ddfc9442da7b7df95f581
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 0. 라이브러리 설정

In [10]:
import ta
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import statsmodels
import warnings
from mplfinance.original_flavor import candlestick2_ohlc

warnings.filterwarnings(action='ignore')
pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.options.display.float_format="{:.5f}".format
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

ModuleNotFoundError: No module named 'ta'